In [1]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [83]:
# Paso 1: Recopilación de datos
# Supongamos que tienes un conjunto de datos con grabaciones de audio y etiquetas correspondientes
df = pd.read_csv('products_with_audio_updated.csv')

In [82]:
# Suponiendo que 'df' es tu DataFrame
for i, _ in enumerate(df['id']):
    df.at[i, 'id'] = i

In [81]:
audio_files = [path for path in df['audio_file2']]
audio_files

['Audios2\\CALIERCORTIN_4miligramos_por_mililitros__50mililitros_iny.mp3',
 'Audios2\\ADTAB_GATO_12miligramos_0.5_2kg_1cp.mp3',
 'Audios2\\ADTAB_GATO_48miligramos_2_8kg_1cp.mp3',
 'Audios2\\NICILAN_200_por_50_120cp.mp3',
 'Audios2\\FORTEKOR_SABOR_2.5miligramos__28cp.mp3',
 'Audios2\\CONOFITE_20mililitros.mp3',
 'Audios2\\OTOCLEAN_LIMPIADOR__OTOLOGICO_18_por_5mililitros.mp3',
 'Audios2\\OXISPEED_60cp.mp3',
 'Audios2\\OTINET_125mililitros.mp3',
 'Audios2\\IRC_VET_60cp.mp3',
 'Audios2\\PAÑO_BUSTER_POLIETILENO__90_por_120cm.mp3',
 'Audios2\\PAÑO_BUSTER_POLIETILENO__60_por_90cm.mp3',
 'Audios2\\PAÑO_BUSTER_POLIETILENO__120_por_120cm.mp3',
 'Audios2\\CANINSULIN_10_por_2.50mililitros_.mp3',
 'Audios2\\GLUCANTIME_5_por_5mililitros.mp3',
 'Audios2\\LUBRITHAL_10gr.mp3',
 'Audios2\\CLEAN_AURAL_CANINO_100mililitros.mp3',
 'Audios2\\T_61_50mililitros_iny.mp3',
 'Audios2\\SCALIBOR_GDE_65cm.mp3',
 'Audios2\\SCALIBOR_PEQ_48cm.mp3',
 'Audios2\\DEXABIOPEN_100mililitros.mp3',
 'Audios2\\DEPO_MODERIN_5mil

In [80]:
labels = [id for id in df['id']]
num_classes = 102

In [85]:
import librosa
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Definir la función para extraer características del audio
def extract_features(audio_path):
    # Cargar el archivo de audio y extraer características
    audio, sr = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return mfccs

# Extraer características de los archivos de audio
X_train = [extract_features(audio_path) for audio_path in audio_files]

max_length = max(mfcc.shape[1] for mfcc in X_train)

def pad_mfcc(mfcc, max_length):
    # Si la longitud de la secuencia es menor que la longitud máxima, rellenar con ceros
    if mfcc.shape[1] < max_length:
        pad_width = max_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant', constant_values=(0,))
    # Si la longitud de la secuencia es mayor que la longitud máxima, trunca la secuencia
    elif mfcc.shape[1] > max_length:
        mfcc = mfcc[:, :max_length]
    return mfcc

# Aplicar la función de relleno/truncado a cada secuencia de MFCCs
X_train_padded = np.array([pad_mfcc(mfcc, max_length) for mfcc in X_train])


X_train_array = np.array(X_train_padded)


In [84]:
y_train = np.array(labels)

In [50]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [51]:
def plot_curves(history):

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']
  train_accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']
  epochs = range(1, len(train_loss) + 1)

  fig, axes = plt.subplots(1, 2, figsize=(14, 5))

  axes[0].plot(epochs, train_loss, 'g-', label='Training Loss')
  axes[0].plot(epochs, val_loss, 'orange', label='Validation Loss')
  axes[0].set_title('Training and Validation Loss')
  axes[0].set_xlabel('Epochs')
  axes[0].set_ylabel('Loss')
  axes[0].legend()
  axes[0].grid(True)

  axes[1].plot(epochs, train_accuracy, 'g-', label='Training Accuracy')
  axes[1].plot(epochs, val_accuracy, 'orange', label='Validation Accuracy')
  axes[1].set_title('Training and Validation Accuracy')
  axes[1].set_xlabel('Epochs')
  axes[1].set_ylabel('Accuracy')
  axes[1].legend()
  axes[1].grid(True)

  plt.tight_layout()

  plt.show()

In [104]:
from keras.callbacks import EarlyStopping

# Aplanar los datos
X_train_array_flat = X_train_array.reshape(X_train_array.shape[0], -1)

model = models.Sequential([
    Dense(150, activation='relu', input_shape=(X_train_array_flat.shape[1],)),
    Dense(150, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Define the optimizer and learning rate
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
num_epochs = 1000
batch_size = 16

# Definir EarlyStopping para detener el entrenamiento cuando la precisión alcance 0.98
#early_stopping = EarlyStopping(monitor='accuracy', min_delta=0, patience=0, verbose=1, mode='max', baseline=0.98)

# Train the model
history = model.fit(X_train_array_flat, y_train, epochs=num_epochs)

# Evaluate the model
loss, accuracy = model.evaluate(X_train_array_flat, y_train)
print("Train Loss:", loss)
print("Train Accuracy:", accuracy)


Epoch 1/1000
4/4 [==============================] - 0s 5ms/step - loss: 157.2473 - accuracy: 0.0000e+00
Epoch 2/1000
4/4 [==============================] - 0s 5ms/step - loss: 78.2652 - accuracy: 0.0392
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 45.0151 - accuracy: 0.0980
Epoch 4/1000
4/4 [==============================] - 0s 5ms/step - loss: 29.2998 - accuracy: 0.0686
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 17.4383 - accuracy: 0.1765
Epoch 6/1000
4/4 [==============================] - 0s 5ms/step - loss: 11.7383 - accuracy: 0.2157
Epoch 7/1000
4/4 [==============================] - 0s 5ms/step - loss: 8.8787 - accuracy: 0.2451
Epoch 8/1000
4/4 [==============================] - 0s 5ms/step - loss: 5.2207 - accuracy: 0.4706
Epoch 9/1000
4/4 [==============================] - 0s 5ms/step - loss: 4.3293 - accuracy: 0.4510
Epoch 10/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.4946 - accuracy: 0.5196
Epoch 11

In [106]:
import pickle

# Guardar el modelo
with open('modelo_entrenado.pkl', 'wb') as f:
    pickle.dump(model, f)
